# Building Your First Image Search with CLIP

In this notebook, you'll learn how to build a simple image search system using CLIP (Contrastive Language-Image Pre-training) from Hugging Face. CLIP can understand both text and images, making it perfect for searching images using natural language descriptions.

## What you'll learn:
- Load a pre-trained CLIP model
- Process images and text
- Find the most similar images to a text query

## Step 1: Helper functions

First, let's define a simple helper function to display images:

In [ ]:
import torch
import torchvision
import torchvision.transforms as T
import matplotlib.pyplot as plt
from PIL import Image
from typing import List

# Only needed on the Udacity workspace. Comment this out if running on another system.
import os
os.environ['HF_HOME'] = '/voc/data/huggingface'
os.environ['OLLAMA_MODELS'] = '/voc/data/ollama/cache'
os.environ['HF_HUB_OFFLINE'] = '1'
os.environ['PATH'] = f"/voc/data/ollama/bin:/voc/data/ffmpeg/bin:{os.environ.get('PATH', '')}"
os.environ['LD_LIBRARY_PATH'] = f"/voc/data/ollama/lib:/voc/data/ffmpeg/lib:{os.environ.get('LD_LIBRARY_PATH', '')}"


def display_grid(images: List[Image.Image], *args, **kwargs):
    """
    Wrap the make_grid function from torchvision accepting PIL.Image instances
    instead of tensors. Every other arguments or keyword arguments is passed
    to make_grid.

    Images are going to be resized and center-cropped to 256x256 to fit
    in the grid.
    """
    transform = T.Compose(
        [
            T.Resize(256),  # resize shorter side to 256
            T.CenterCrop(256),  # center crop to 256x256 square
            T.ToTensor(),
        ]
    )

    tensors = [transform(img) for img in images]
    grid = torchvision.utils.make_grid(tensors, *args, **kwargs)
    _, sub = plt.subplots(dpi=300)
    sub.imshow(grid.permute(1, 2, 0))
    sub.axis("off")
    plt.show()

## Step 2: Import Libraries and Load the Model

Now let's import the necessary libraries and load a pre-trained CLIP model from Hugging Face.

In [ ]:
from transformers import CLIPProcessor, CLIPModel

# Load the CLIP model and processor
model_name = "openai/clip-vit-base-patch32"

# TODO: load processor and model using .from_pretrained
# on respectively CLIPProcessor and CLIPModel
processor = ... # complete
model = ... # complete

print("Model loaded successfully!")

## Step 3: Load Dataset

Let's load a dataset of images:

In [ ]:
from datasets import load_dataset
from tqdm import tqdm

# TODO: Load the "test" split of the "xai-org/RealworldQA" dataset
# We use this Realworld Q/A dataset because it is small (765) but
# pretty diverse
dataset = ... # complete

In [ ]:
# Let's look at what we got. Let's display a few images in a grid
images = dataset['image']
display_grid(images[:25], nrow=5)

## Step 4: Create Image Embeddings

Now we'll convert our images into embeddings (numerical representations) that CLIP can work with. These embeddings capture the visual features of each image.

In [ ]:
import numpy as np


# Process images and get embeddings
def get_image_embeddings_batch(images, batch_size=32):
    """
    Process images in batches for speed
    """

    all_embeddings = []

    for i in tqdm(
        range(0, len(images), batch_size), total=int(np.ceil(len(images) / batch_size))
    ):
        
        # Get a batch of images
        batch = images[i : i + batch_size]

        # TODO: Run the pre-processor to get the inputs to the CLIP model
        # Remember to use return_tensors="pt" and padding=True
        inputs = ... # complete

        # TODO: Get image features from the model using get_image_features
        with torch.no_grad():
            # HINT: remember to unpack the inputs with **inputs
            image_features = ... # complete

        # NOTE: since we will be using cosine similarity, we pre-normalize the embeddings
        image_features = image_features / image_features.norm(dim=-1, keepdim=True)

        all_embeddings.append(image_features)

        if (i + batch_size) % 100 == 0 or (i + batch_size) >= len(images):
            print(f"Processed {min(i + batch_size, len(images))} images...")
    
    # Right now the list all_embeddings is a list of batches, we
    # collapse it into a list of embeddings
    return torch.cat(all_embeddings, dim=0)


print("Creating embeddings for all batches of images...")
image_embeddings = get_image_embeddings_batch(images)
print(f"Created embeddings with shape: {image_embeddings.shape}")

## Step 5: Search Images with Images

Here you will implement an image-to-image search. Given a query image, the function should:

1. Compute the embedding of the query image using .get_image_features
2. Normalize the image feature
3. Compute the cosine similarity. Since the query image features and the
    dataset embeddings are already normalized, cosine similarity is now 
    equivalent to just the matrix multiplication between the query image embeddings
    and the image embeddings transposed.
4. sort the image similarities in descending order and take the first top_k elements
5. return the scores and the top_indices

In [ ]:
def search_images(
    image_embeddings: List[torch.Tensor],
    query_image: Image.Image,
    top_k: int = 5,
    plot: bool = False,
):
    """
    Given a set of image embeddings and an image query, returns the first top_k matches
    with the highest cosine-similarity scores. 

    The image embeddings are assumed pre-normalized.
    """

    # TODO: Pre-process the input image
    # Remember to use return_tensors="pt" and padding=True
    # HINT: provide to the processor a list of one element: [query_image]
    inputs = ... # complete

    # Get image embedding from the model
    with torch.no_grad():
        # TODO: use get_image_features.
        # HINT: remember to unpack the inputs with **inputs
        query_image_features = ... # complete

    print(
        f"Query image has been transformed to embedding vector of shape {query_image_features[0].shape} ..."
    )

    # We use cosine similarity as matching score:
    # cosine_similarity = (A · B) / (||A|| × ||B||)
    #  so we need to:
    # 1 - Normalize query image features
    query_image_features = query_image_features / query_image_features.norm(dim=-1, keepdim=True)

    # 2 - Normalize image embeddings
    # (we already did during pre-computation!)

    # 3 - Now the formula reduces to just the matrix multiplication
    # TODO: do the matrix multiplication. Remember to transpose image_embeddings
    # HINT: use torch.matmul on the query image features and the transpose of the image
    # embeddings
    similarities = ... # complete

    # Sort the similarities and get the _indices_ of the first "top_k" results
    top_indices = similarities[0].argsort(descending=True)[:top_k]

    scores = similarities[0][top_indices]

    if plot:

        for i, score in zip(top_indices, scores):
            print(f"{i} - {score:.3f}")

        display_grid([images[int(i)] for i in top_indices], nrow=5)

    return top_indices, scores

## Step 6: Try Some Searches

Now let's test our image-to-image search system:

In [ ]:
# Let's an image of a street and see if we find others:
street_image = images[222]
street_image

In [ ]:
indices, scores = search_images(image_embeddings, street_image, top_k=5, plot=True)

> **TODO**: Can you explain why the first image is exactly the one we provided as query and has a score of 1?

(Since the image we give is part of the dataset, the first result is bound to be that image with a perfect score, because an image is a perfect match to itselfQ)

In [ ]:
query_image = images[0]
query_image

In [ ]:
# TODO: run the image-to-image search again as we did previously
... # complete